In [ ]:
import heapq
import copy
import numpy as np
import itertools
from data_preprocessing import preprocessing, DUMMY_PENALTY

SEASON = "Season_6.json"

def evaluate_matching(candidate, ceremonies):
    total_cost = 0
    candidate_pairs = {(man, candidate[man]) for man in candidate}
    for ceremony in ceremonies:
        ceremony_pairs = {(p['man'].lower(), p['woman'].lower()) for p in ceremony['pairs']}
        observed = ceremony['score']
        matching_count = sum(
            1 for (man, woman) in candidate_pairs
            if not man.startswith("dummy") and not woman.startswith("dummy") and (man, woman) in ceremony_pairs
        )
        total_cost += abs(matching_count - observed)
    for man, woman in candidate.items():
        if man.startswith("dummy") or woman.startswith("dummy"):
            total_cost += DUMMY_PENALTY
    return total_cost

def partial_cost(state, ceremonies):
    cost = 0
    candidate_pairs = {(man, state[man]) for man in state}
    for ceremony in ceremonies:
        ceremony_pairs = {(p['man'].lower(), p['woman'].lower()) for p in ceremony['pairs']}
        observed = ceremony['score']
        current_matches = sum(
            1 for (man, woman) in candidate_pairs
            if not man.startswith("dummy") and not woman.startswith("dummy") and (man, woman) in ceremony_pairs
        )
        if current_matches > observed:
            cost += (current_matches - observed)
    for man, woman in state.items():
        if man.startswith("dummy") or woman.startswith("dummy"):
            cost += DUMMY_PENALTY
    return cost

def heuristic(state, ceremonies, remaining_count):
    h = 0
    candidate_pairs = {(man, state[man]) for man in state}
    for ceremony in ceremonies:
        ceremony_pairs = {(p['man'].lower(), p['woman'].lower()) for p in ceremony['pairs']}
        observed = ceremony['score']
        current_matches = sum(
            1 for (man, woman) in candidate_pairs
            if not man.startswith("dummy") and not woman.startswith("dummy") and (man, woman) in ceremony_pairs
        )
        if current_matches >= observed:
            h += (current_matches - observed)
        else:
            best_possible = current_matches + remaining_count
            h += max(0, observed - best_possible)
    return h

def a_star_search(confirmed, variable_men, participants, fixed, ceremonies):
    # Verwende einen Counter als Tie-Breaker
    counter = itertools.count()
    
    initial_state = confirmed.copy()
    initial_remaining = variable_men.copy()
    g = partial_cost(initial_state, ceremonies)
    h = heuristic(initial_state, ceremonies, len(initial_remaining))
    f = g + h
    
    # Tupel: (f, g, counter, state, remaining)
    frontier = [(f, g, next(counter), initial_state, initial_remaining)]
    
    best_complete = None
    best_complete_cost = float('inf')
    
    while frontier:
        f, g, _, state, remaining = heapq.heappop(frontier)
        if not remaining:
            total_cost = evaluate_matching(state, ceremonies)
            if total_cost < best_complete_cost:
                best_complete_cost = total_cost
                best_complete = state
            continue
        next_man = remaining[0]
        new_remaining = remaining[1:]
        used_women = set(state.values())
        allowed = [w for w in participants['women'] if w not in used_women and not fixed.get((next_man, w), False)]
        for woman in allowed:
            new_state = state.copy()
            new_state[next_man] = woman
            new_g = partial_cost(new_state, ceremonies)
            new_h = heuristic(new_state, ceremonies, len(new_remaining))
            new_f = new_g + new_h
            heapq.heappush(frontier, (new_f, new_g, next(counter), new_state, new_remaining))
    return best_complete, best_complete_cost

if __name__ == "__main__":
    # Preprocessing durchführen und Daten laden
    season = SEASON
    probabilities, fixed, participants, ceremonies = preprocessing(season)
    
    confirmed = {}
    for man in participants['men']:
        for woman in participants['women']:
            if fixed.get((man, woman), False) and probabilities.get((man, woman), 0) == 1.0:
                confirmed[man] = woman
                break
    variable_men = [man for man in participants['men'] if man not in confirmed]
    
    best_matching, best_cost = a_star_search(confirmed, variable_men, participants, fixed, ceremonies)
    
    if best_matching is not None:
        print("\nDas durch A*-Suche gefundene optimale Matching ist:")
        for man, woman in sorted(best_matching.items()):
            print(f"{man.capitalize()} - {woman.capitalize()}", end=" | ")
        print(f"\nGesamtkosten: {best_cost}")
    else:
        print("Kein vollständiges Matching gefunden.")


[Ceremony 2] Blackout triggered!
[Ceremony 4] Blackout triggered!
[Ceremony 8] Blackout triggered!
[Ceremony 9] Blackout triggered!

Das durch A*-Suche gefundene optimale Matching ist:
Aaron - Melissa | Dario - Jill | Dominik - Kathleen | Dummy_man_1 - Victoria | Germain - Christin | Marc - Mirjam | Marcel - Leonie | Marko - Vanessa_e | Marvin - Sabrina | Maximilian - Vanessa_m | Sascha - Laura | 
Gesamtkosten: 13
